In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46860
2,Huelva,Confirmados PDIA 14 días,570
3,Huelva,Tasa PDIA 14 días,"111,07430286259914"
4,Huelva,Confirmados PDIA 7 días,291
5,Huelva,Tasa PDIA 7 dias,"56,70635461932693"
6,Huelva,Total Confirmados,47063
7,Huelva,Curados,45565
8,Huelva,Fallecidos,422


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46860.0


/tmp/ipykernel_4755/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12929.0


/tmp/ipykernel_4755/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4755/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4755/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4755/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1151 personas en los últimos 7 días 

Un positivo PCR cada 678 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46860.0,291.0,570.0,513170.0,56.706355,111.074303,67.0
Huelva-Costa,27706.0,182.0,326.0,289548.0,62.856590,112.589277,40.0
Huelva (capital),12929.0,125.0,212.0,143837.0,86.903926,147.389058,27.0
Condado-Campiña,14595.0,98.0,207.0,156231.0,62.727628,132.496112,26.0
Palos de la Frontera,1136.0,26.0,58.0,11742.0,221.427355,493.953330,6.0
Cartaya,2200.0,33.0,55.0,20083.0,164.318080,273.863467,6.0
Moguer,2070.0,20.0,31.0,21867.0,91.462020,141.766132,5.0
Bollullos Par del Condado,1333.0,13.0,35.0,14387.0,90.359352,243.275179,4.0
Almonte,2360.0,9.0,28.0,24507.0,36.724201,114.253071,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palos de la Frontera,1136.0,26.0,58.0,11742.0,221.427355,493.953330,6.0
Granada de Río-Tinto (La),6.0,1.0,1.0,243.0,411.522634,411.522634,0.0
Cartaya,2200.0,33.0,55.0,20083.0,164.318080,273.863467,6.0
Bollullos Par del Condado,1333.0,13.0,35.0,14387.0,90.359352,243.275179,4.0
San Bartolomé de la Torre,395.0,5.0,7.0,3761.0,132.943366,186.120713,0.0
Palma del Condado (La),1454.0,10.0,18.0,10801.0,92.584020,166.651236,3.0
Huelva (capital),12929.0,125.0,212.0,143837.0,86.903926,147.389058,27.0
Paterna del Campo,216.0,2.0,5.0,3457.0,57.853630,144.634076,1.0
Moguer,2070.0,20.0,31.0,21867.0,91.462020,141.766132,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,863.0,1.0,4.0,12750.0,7.843137,31.372549,-1.0,0.250000
Sierra de Huelva-Andévalo Central,4131.0,10.0,35.0,67391.0,14.838777,51.935718,1.0,0.285714
Almonte,2360.0,9.0,28.0,24507.0,36.724201,114.253071,4.0,0.321429
Alosno,235.0,1.0,3.0,3933.0,25.425884,76.277651,1.0,0.333333
Aracena,677.0,1.0,3.0,8255.0,12.113870,36.341611,0.0,0.333333
Punta Umbría,1281.0,1.0,3.0,15355.0,6.512537,19.537610,0.0,0.333333
Isla Cristina,3194.0,9.0,26.0,21393.0,42.069836,121.535082,3.0,0.346154
Bollullos Par del Condado,1333.0,13.0,35.0,14387.0,90.359352,243.275179,4.0,0.371429
Paterna del Campo,216.0,2.0,5.0,3457.0,57.853630,144.634076,1.0,0.400000
